In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [34]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Exclude examples labeled 'N' or 'Done'
df_filtered = df[(df['label'] != 'N') & (df['label'] != 'Done')]

# Extract features and labels
X = df_filtered.iloc[:, 2:9].values  # assuming your features start from the 3rd column
y = df_filtered['label'].values

In [35]:
# Specify the order of your classes
class_order = ['L', 'R']  # Adjust this based on your actual class order

# Initialize and fit the LabelEncoder with specified classes
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

In [36]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Reshape the data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [41]:
# Build the CNN model with two convolutional layers
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
print("Shape after the first convolutional layer:", model.output_shape)

model.add(MaxPooling1D(pool_size=2))
print("Shape after the first convolutional layer:", model.layers[1].input_shape)
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so 1 neuron with sigmoid activation

Shape after the first convolutional layer: (None, 5, 32)
Shape after the first convolutional layer: (None, 5, 32)


In [42]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6253 - accuracy: 0.6860 - val_loss: 0.6177 - val_accuracy: 0.6742
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.6051 - accuracy: 0.6885 - val_loss: 0.6061 - val_accuracy: 0.6742
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5990 - accuracy: 0.6885 - val_loss: 0.5997 - val_accuracy: 0.6742
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5950 - accuracy: 0.6885 - val_loss: 0.5950 - val_accuracy: 0.6742
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5942 - accuracy: 0.6860 - val_loss: 0.5902 - val_accuracy: 0.6742
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5904 - accuracy: 0.6885 - val_loss: 0.5969 - val_accuracy: 0.6742
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.5833 - accuracy: 0.6873 - val_loss: 0.5836 - val_accuracy: 0.6742
Epoch 8/10
25/25 [==

In [44]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 957us/step
Test Accuracy: 0.6326530612244898
